In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from scipy import misc
import sys
import os
import tensorflow as tf
import numpy as np
import facenet
import align.detect_face
import random
from time import sleep
import cv2

import math
import pickle
from sklearn.svm import SVC

In [3]:
margin = 44
gpu_memory_fraction = 1.0
detect_multiple_faces = False

In [4]:
seed = 666
image_size = 160
do_prewhiten = True
model = '/home/topica/workspace/facenet/src/ckpt/20180402-114759/'
classifier_filename = '/home/topica/workspace/facenet/src/model_topica_weight.pkl'
class_name = ['0','1','2','3','4','5','6','7','8','9','10']
# image_path = '/home/topica/workspace/facenet/dataset_topica/0/face_2_334.png'

In [5]:
sleep(random.random())

print('Creating networks and loading parameters')

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)

minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor



Creating networks and loading parameters


In [6]:
 with tf.Graph().as_default():
        with tf.Session() as sess:   
            np.random.seed(seed=seed)
            
            # Load the model
            print('Loading feature extraction model')
            facenet.load_model(model)
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            
            classifier_filename_exp = os.path.expanduser(classifier_filename)
            
            with open(classifier_filename_exp, 'rb') as infile:
                (model, class_names) = pickle.load(infile)
            print(class_names)
            cap = cv2.VideoCapture("/home/topica/Video_topica/T6_08_20180816_090000.avi")
            frame = 0
            while(True):
                ret, img = cap.read()
        
#                 have_face, crop_face, bb = crop_face(img)
                # crop face
                bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
                nrof_faces = bounding_boxes.shape[0]
                if nrof_faces>0:
                    have_face = True
                    det = bounding_boxes[:,0:4]
                    det_arr = []
                    result_face = []
                    img_size = np.asarray(img.shape)[0:2]
                    if nrof_faces>1:
                        if detect_multiple_faces:
                            for i in range(nrof_faces):
                                det_arr.append(np.squeeze(det[i]))
                        else:
                            bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                            img_center = img_size / 2
                            offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                            offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                            index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                            det_arr.append(det[index,:])
                    else:
                        det_arr.append(np.squeeze(det))

                    for i, det in enumerate(det_arr):
                        det = np.squeeze(det)
                        bb = np.zeros(4, dtype=np.int32)
                        bb[0] = np.maximum(det[0]-margin/2, 0)
                        bb[1] = np.maximum(det[1]-margin/2, 0)
                        bb[2] = np.minimum(det[2]+margin/2, img_size[1])
                        bb[3] = np.minimum(det[3]+margin/2, img_size[0])
                        cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                        scaled = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
                #             result_face.append(scaled)
                else:
                    have_face = False
               
                if (have_face == True):
                    cv2.rectangle(img,(bb[0],bb[1]),(bb[2],bb[3]),(64,64,64),2) #highlight detected face
                    # Run forward pass to calculate embeddings
                    emb_array = np.zeros((1, embedding_size))
                        # prewhiten
                    if do_prewhiten:
                        scaled = facenet.prewhiten(scaled)
                    scaled = np.reshape(scaled,(1, image_size, image_size,3))
                        # feed_dict
                    feed_dict = { images_placeholder:scaled, phase_train_placeholder:False }
                    emb_array[0:1,:] = sess.run(embeddings, feed_dict=feed_dict)

                    # Classify images
                    predictions = model.predict_proba(emb_array)
                    best_class_indices = np.argmax(predictions, axis=1)
                    best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                    result = str(class_names[best_class_indices[0]]) + ": " + str(best_class_probabilities[0])
#                     print(' %s: %.3f' % (class_names[best_class_indices[0]], best_class_probabilities[0]))
                    color = (255,255,255)
                    cv2.putText(img, result, (int(bb[0]+5), int(bb[1]-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                cv2.imshow('img',img)

                frame = frame + 1

                #---------------------------------
                if cv2.waitKey(70) & 0xFF == ord('q'): #press q to quit
                    break
            cv2.destroyAllWindows()

Loading feature extraction model
Model directory: /home/topica/workspace/facenet/src/ckpt/20180402-114759/
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from /home/topica/workspace/facenet/src/ckpt/20180402-114759/model-20180402-114759.ckpt-275
['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']


/home/topica/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
